In [32]:
import os, sys

sys.path.append(
    os.path.dirname(
            os.path.realpath(
                ''
            )
    )
)

import datetime
import pandas as pd
from scrapping_sources.Nasdaq import Nasdaq
from scrapping_sources.Macrotrend import Macrotrend
from models import Security, Base
from config import DATABASE_URI
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine(DATABASE_URI)
session = sessionmaker(bind=engine)

s = session()

N = Nasdaq()
M = Macrotrend()

statements = [
'income_statement',
'balance_sheet',
'cash_flow_statement',
'financial_ratios'
]
time_format = [
    'annual',
    'quarterly'
]


def convert_date_N(date):
    """Converts the data from Nasdaq scrapper to same format"""
    return datetime.datetime.strptime(str(date), '%b/%Y').strftime("%Y-%m")

def convert_to_ending_period_format(date):
    """Converts the data from Database scrapper to same format"""
    result = datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime("%Y-%m")
    return result

# WAY TOO SLOW!
def last_period_db(ticker):
    """Returns the the latest date available for statements in Database """

    try:
        period_list_on_db = []

        for stmnt in statements:
            for t_format in time_format:

                r = s.query(Base.metadata.tables[f'{stmnt}_{t_format}'].columns['date']).where(
                    Base.metadata.tables[f'{stmnt}_{t_format}'].columns['ticker'] == str(ticker)
                ).all()
                r.sort(reverse=True)
                r =  str(r[0][0])
                period_list_on_db.append(r)

        r = max(period_list_on_db)

        return convert_to_ending_period_format(r)
    
    except:
    
        return None

# REPLACE WITH DATETUIME.DATE.TODAY()
tickers = N.earnings_release_tickers(date='2021-10-15')

if tickers is not None:


    df_earnings = pd.DataFrame()
    df_earnings['date'] = [datetime.date.today()] * len(tickers)
    df_earnings['ticker'] = tickers

    querry = s.query(Security.ticker, Security.id).all()
    security_map = {querry[i][0]: querry[i][1] for i, v in zip(range(len(querry)), range(len(querry)))}

    df_earnings['security_id'] = df_earnings['ticker'].map(security_map)

    data = N.earnings_release('2021-10-15')
    df_earnings['last_period_N'] = data['fiscalQuarterEnding'].map(convert_date_N)
    df_earnings['last_period_DB'] = df_earnings['ticker'].map(last_period_db)
    print(df_earnings)
    df_earnings['last_period_M'] = df_earnings['ticker'].map(M.latest_ending_period_available)

    print(df_earnings)
#     try:
#         df_earnings.to_sql(con=engine, name="earnings_release", index=False, if_exists="append")
#     except:
#         print("earnings table is already up to date")

# else:
#     print('No data to collect today')
#     pass

s.close_all()

          date ticker  security_id last_period_N last_period_DB
0   2021-10-16    TSM         7466       2021-09        2021-06
1   2021-10-16    UNH         7625       2021-09        2021-06
2   2021-10-16     PM         5873       2021-09        2021-06
3   2021-10-16   SCHW         6571       2021-09        2021-06
4   2021-10-16      C         1142       2021-09        2021-06
5   2021-10-16    AMD          330       2021-09        2021-06
6   2021-10-16     GS         3275       2021-09        2021-06
7   2021-10-16    USB         7663       2021-09        2021-06
8   2021-10-16     BX         1121       2021-09        2021-06
9   2021-10-16    TFC         7276           NaN        2021-06
10  2021-10-16    SLB         6771           NaN        2021-06
11  2021-10-16    PPG         5925           NaN        2021-06
12  2021-10-16    FRC         2871           NaN        2021-06
13  2021-10-16    KEY         4257           NaN        2021-06
14  2021-10-16    WAL         8003      

/tmp/ipykernel_6346/817923800.py:104: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [31]:
for ticker in tickers:
    print(last_period_db(ticker))


2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-06
2021-05
2021-06
2021-05
2021-06
2021-05
2021-06
2021-06
2021-04
2021-06
2021-05
2021-06
2021-05
2021-06


In [19]:
r = s.query(Base.metadata.tables[f'balance-sheet-annual'].columns['date']).where(
    Base.metadata.tables[f'{stmnt}_{t_format}'].columns['ticker'] == str(ticker)
).all()
r.sort(reverse=True)
r =  str(r[0][0])


TypeError: object of type 'NoneType' has no len()

In [58]:

def get_latest_in_db(tickers: tuple ):



    r = engine.execute(f"""
    SELECT ticker, MAX(date)
    FROM public.balance_sheet_annual
    WHERE ticker IN {tickers}
    GROUP BY ticker""")

    for i in r:
        print(i)


In [59]:
get_latest_in_db(('AAPL','MSFT'))

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "f"
LINE 4:     WHERE ticker IN f('AAPL', 'MSFT')
                            ^

[SQL: 
    SELECT ticker, MAX(date)
    FROM public.balance_sheet_annual
    WHERE ticker IN f('AAPL', 'MSFT')
    GROUP BY ticker]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [25]:
s.close_all()

/tmp/ipykernel_6346/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()
